In [82]:
import torch
import glob
import os
import pickle
from rouge import Rouge
import random

In [59]:
def _get_ngrams(n, text):
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

def _block_tri(c, p):
    tri_c = _get_ngrams(3, c.split())
    for s in p:
        tri_s = _get_ngrams(3, s.split())
        if len(tri_c.intersection(tri_s))>0:
            return True
    return False

In [10]:
dataname = "infer_all.pkl"
with open(dataname, 'rb') as file:
    data = pickle.load(file)

In [11]:
print(data.keys())


dict_keys(['src_txt', 'tgt_txt', 'src_sent_labels', 'scores'])


In [6]:
print(data['src_txt'][0])

['a university of iowa student has died nearly three months after a fall in rome in a suspected robbery attack in rome .', 'andrew mogni , 20 , from glen ellyn , illinois , had only just arrived for a semester program in italy when the incident happened in january .', 'he was flown back to chicago via air ambulance on march 20 , but he died on sunday .', 'andrew mogni , 20 , from glen ellyn , illinois , a university of iowa student has died nearly three months after a fall in rome in a suspected robbery', 'he was taken to a medical facility in the chicago area , close to his family home in glen ellyn .', "he died on sunday at northwestern memorial hospital - medical examiner 's office spokesman frank shuftan says a cause of death wo n't be released until monday at the earliest .", 'initial police reports indicated the fall was an accident but authorities are investigating the possibility that mogni was robbed .', "on sunday , his cousin abby wrote online : ` this morning my cousin andr

In [12]:
print(data['src_sent_labels'][0])

[0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [13]:
data_list = [dict(zip(data,t)) for t in zip(*data.values())]

In [14]:
print(data_list[0].keys())

dict_keys(['src_txt', 'tgt_txt', 'src_sent_labels', 'scores'])


### Evaluate top-k sentence selection

In [63]:
def get_topk_selection(dataset, num_examples,k = 3, ngram=3):
    hyps = []
    refs = []
    for i in range(min(len(dataset),num_examples)):
        example = dataset[i]
        selected = []
        scores = example['scores']
        scores = [(score, i) for i, score in enumerate(scores)]
        scores_sorted = sorted(scores)
        ngram_set = set()
        count = 0
        while scores_sorted and count < k:
            _, cand_idx = scores_sorted.pop()
            cand_ngram = _get_ngrams(ngram, example['src_txt'][cand_idx])
            if len(ngram_set.intersection(cand_ngram)) == 0:
                ngram_set.update(cand_ngram)
                selected.append(example['src_txt'][cand_idx])
                count += 1
        if not selected:
            continue
        out = " ".join(selected)
        hyps.append(out)
        ref = example['tgt_txt'].replace('<q>',' ')
        refs.append(ref)

    return hyps, refs

In [77]:
hyps, refs = get_topk_selection(data_list, 100000)

In [67]:
print(len(hyps), len(refs))

1000 1000


In [78]:
rouge_metric = Rouge()

rouge_metric.get_scores(hyps, refs, avg=True)

{'rouge-1': {'r': 0.2761579258224892,
  'p': 0.5042588714386943,
  'f': 0.34527010526606833},
 'rouge-2': {'r': 0.11644383704083192,
  'p': 0.23932371326885649,
  'f': 0.15012067596676093},
 'rouge-l': {'r': 0.211787900163177,
  'p': 0.3893834698036595,
  'f': 0.2653227577647347}}

### validate oracle baseline

In [69]:
def get_oracle_selection(dataset, num_examples):
    hyps = []
    refs = []
    for i in range(min(len(dataset),num_examples)):
        example = dataset[i]
        selected = []
        for idx, j in enumerate(example['src_sent_labels']):
            if j == 1:
                selected.append(example['src_txt'][idx])
        if not selected:
            continue
        out = " ".join(selected)
        hyps.append(out)
        ref = example['tgt_txt'].replace('<q>',' ')
        refs.append(ref)

    return hyps, refs
        

In [71]:
hyps_oracle, refs_oracle = get_oracle_selection(data_list, 1000)


In [72]:
print(len(hyps_oracle))

986


In [73]:
print(len(refs_oracle))

986


In [76]:
rouge_metric = Rouge()
rouge_metric.get_scores(hyps_oracle, refs_oracle, avg=True)

{'rouge-1': {'r': 0.5676532245422679,
  'p': 0.5775291688161241,
  'f': 0.5508185184090183},
 'rouge-2': {'r': 0.3268545562111377,
  'p': 0.33508357536518946,
  'f': 0.31426476241945256},
 'rouge-l': {'r': 0.4685939421781878,
  'p': 0.4776104916914664,
  'f': 0.4552125771451799}}

### Evaluate random-3 selection

In [83]:
def get_random_k_selection(dataset, num_examples,k = 3, ngram=3):
    hyps = []
    refs = []
    for i in range(min(len(dataset),num_examples)):
        example = dataset[i]
        selected = []
        scores = example['scores']
        scores = [(score, i) for i, score in enumerate(scores)]
        random.shuffle(scores)
        ngram_set = set()
        count = 0
        while scores and count < k:
            _, cand_idx = scores.pop()
            cand_ngram = _get_ngrams(ngram, example['src_txt'][cand_idx])
            if len(ngram_set.intersection(cand_ngram)) == 0:
                ngram_set.update(cand_ngram)
                selected.append(example['src_txt'][cand_idx])
                count += 1
        if not selected:
            continue
        out = " ".join(selected)
        hyps.append(out)
        ref = example['tgt_txt'].replace('<q>',' ')
        refs.append(ref)

    return hyps, refs

In [84]:
hyps_random, refs_random = get_random_k_selection(data_list, 100000)

In [85]:
rouge_metric = Rouge()
rouge_metric.get_scores(hyps_random, refs_random, avg=True)

{'rouge-1': {'r': 0.16829301833168941,
  'p': 0.3351070189848325,
  'f': 0.21465348193063072},
 'rouge-2': {'r': 0.04291650292877776,
  'p': 0.09543145621139532,
  'f': 0.056200722933811544},
 'rouge-l': {'r': 0.12336082609000679,
  'p': 0.2490458299325844,
  'f': 0.1579106354309573}}

### validate lead 3 baseline

In [86]:
def get_lead_k_selection(dataset, num_examples, k=3):
    hyps = []
    refs = []
    for i in range(min(len(dataset),num_examples)):
        example = dataset[i]
        selected = []
        for j in range(min(k, len(example['src_sent_labels']))):
            selected.append(example['src_txt'][j])
        out = " ".join(selected)
        hyps.append(out)
        ref = example['tgt_txt'].replace('<q>',' ')
        refs.append(ref)

    return hyps, refs

In [88]:
hyps_lead, refs_lead = get_lead_k_selection(data_list, 100000)
rouge_metric.get_scores(hyps_lead, refs_lead, avg=True)

{'rouge-1': {'r': 0.5084508230373855,
  'p': 0.3589632929014358,
  'f': 0.4097730984074396},
 'rouge-2': {'r': 0.22330570331577615,
  'p': 0.14606575470871974,
  'f': 0.17037691800181165},
 'rouge-l': {'r': 0.40554740519038524,
  'p': 0.2855664228268964,
  'f': 0.3262607094733872}}